# Filtering

This notebook is for making jsons from the data.

In [9]:
from IPython.display import display
import pandas as pd

In [5]:
df_htags = pd.read_pickle('../data/interim/hashtag_indices.pkl')
df = pd.read_pickle('../data/interim/aggregated.pkl')

In [6]:
def make_gen(key, series):
    for i, keys in series.iteritems():
        if key in keys:
            yield i

In [7]:
df_mps = pd.read_excel('../data/raw/Hackathon_WomenMP.xlsx')
df_mps['twitter_username'] = df_mps['twitter_username'].map(lambda x: '@' + x, na_action='ignore')
df_mps.head()

,name,sort_name,twitter_username,email,party,constituency,followers,Race/ Ethicity,Ethnic minority,Religion minority,Age/ DOB,image
0,Alison McGovern,"McGovern, Alison",@Alison_McGovern,alison.mcgovern.mp@parliament.uk,Labour,Wirral South,28505,NaN,White,NaN,1980,https://candidates.democracyclub.org.uk/media/...
1,Alison Thewliss,"Thewliss, Alison",@alisonthewliss,alison.thewliss.mp@parliament.uk,Scottish National Party,Glasgow Central,13818,NaN,White,NaN,1982,https://candidates.democracyclub.org.uk/media/...
2,Amanda Milling,"Milling, Amanda",@amandamilling,amanda.milling.mp@parliament.uk,Conservative,Cannock Chase,3961,NaN,White,NaN,1975,https://candidates.democracyclub.org.uk/media/...
3,Amanda Solloway,"Solloway, Amanda",NaN,amanda.solloway.mp@parliament.uk,Conservative,Derby North,0,NaN,White,NaN,1961,https://candidates.democracyclub.org.uk/media/...
4,Amber Rudd,"Rudd, Amber",@AmberRuddHR,amber.rudd.mp@parliament.uk,Conservative,Hastings and Rye,39798,NaN,White,NaN,1963,https://candidates.democracyclub.org.uk/media/...


In [8]:
df_mps['Tweeted At'] = df_mps['twitter_username'].map(lambda x: [i for i in make_gen(x, df['Tweeted At'])])
df_mps.head()

,name,sort_name,twitter_username,email,party,constituency,followers,Race/ Ethicity,Ethnic minority,Religion minority,Age/ DOB,image,Tweeted At
0,Alison McGovern,"McGovern, Alison",@Alison_McGovern,alison.mcgovern.mp@parliament.uk,Labour,Wirral South,28505,NaN,White,NaN,1980,https://candidates.democracyclub.org.uk/media/...,"[229, 232, 416, 827, 962, 1320, 2735, 3018, 31..."
1,Alison Thewliss,"Thewliss, Alison",@alisonthewliss,alison.thewliss.mp@parliament.uk,Scottish National Party,Glasgow Central,13818,NaN,White,NaN,1982,https://candidates.democracyclub.org.uk/media/...,"[2, 205, 434, 860, 1445, 2033, 2167, 2191, 250..."
2,Amanda Milling,"Milling, Amanda",@amandamilling,amanda.milling.mp@parliament.uk,Conservative,Cannock Chase,3961,NaN,White,NaN,1975,https://candidates.democracyclub.org.uk/media/...,"[23689, 24296, 24523, 24672, 24691, 24739, 247..."
3,Amanda Solloway,"Solloway, Amanda",NaN,amanda.solloway.mp@parliament.uk,Conservative,Derby North,0,NaN,White,NaN,1961,https://candidates.democracyclub.org.uk/media/...,[]
4,Amber Rudd,"Rudd, Amber",@AmberRuddHR,amber.rudd.mp@parliament.uk,Conservative,Hastings and Rye,39798,NaN,White,NaN,1963,https://candidates.democracyclub.org.uk/media/...,"[0, 9, 10, 22, 48, 80, 85, 88, 97, 105, 108, 1..."


In [44]:
df_htags.head()

,Hashtag,Indices,Count
0,#GE2017:,"[1, 92, 443, 445, 664, 1002, 1135, 1441, 1459,...",161
1,#GeneralElection17,"[2, 131, 159, 1068, 1161, 1336, 1363, 1814, 18...",195
2,#GE2107,"[2, 437, 816, 881, 1042, 1191, 1217, 1281, 128...",295
3,#UKelections2017,"[3, 6227]",2
4,#GE2017,"[9, 10, 15, 17, 38, 48, 49, 50, 60, 63, 70, 77...",48761


In [67]:
from functools import reduce

def filter_handle(handles, query='GUID >= 0'):
    index_list = df_mps[df_mps['twitter_username'].isin(['@' + handle for handle in handles])]['Tweeted At'].values
    indices = reduce(lambda x, y : set(x) | set(y), index_list)
    return df.loc[indices, :].query(query)
    
def filter_tag(tags, query='GUID >= 0'):
    index_list = df_htags[df_htags['Hashtag'].isin(['#' + tag for tag in tags])]['Indices'].values
    indices = reduce(lambda x, y : set(x) | set(y), index_list)
    return df.loc[indices, :].query(query)

In [66]:
handles = ['alisonthewliss']
len(filter_handle(handles))

28717

In [53]:
len(filter_handle(['HackneyAbbott'])['Contents'])

277115

In [68]:
filter_tag(['GE2017', 'JC4PM'], 'Followers > 30000')

,GUID,Date (GMT),URL,Contents,Author,Name,Country,State/Region,City/Urban Area,Category,Emotion,Source,Klout Score,Gender,Posts,Followers,Following,Tweeted At,Hashtags
15,872966000723013632,2017-06-08 23:58:00,http://twitter.com/iamthefaizan/status/8729660...,@iihtishamm Congrats @NazShahBfd #GE2017,@iamthefaizan,?????¡?¡ ???za????,NaN,NaN,NaN,NaN,NaN,Twitter,59.0,NaN,91820.0,34794.0,841.0,"[@iihtishamm, @NazShahBfd]",[#GE2017]
185,872962241804599299,2017-06-08 23:43:00,http://twitter.com/iihtishamm/status/872962241...,Congratulations @NazShahBfd from Bradford ...!...,@iihtishamm,Ihtisham ul Haq,United Kingdom,Greater London,London,NaN,NaN,Twitter,64.0,NaN,17286.0,69735.0,531.0,[@NazShahBfd],"[#JeremyCorbyn4PM, #GE2017]"
328,872958459716788225,2017-06-08 23:28:00,http://twitter.com/Injury_Claims_/status/87295...,"RT @Gian_TCatt No @bbclaurak ""the scalp"" of @A...",@Injury_Claims_,Daniel Blake's sista,Mexico,Distrito Federal,Mexico City,NaN,NaN,Twitter,59.0,M,76820.0,40460.0,33423.0,"[@Gian_TCatt, @bbclaurak, @AmberRuddHR, @nw_ni...",[#GE2017]
380,872957600194801664,2017-06-08 23:24:00,http://twitter.com/BBCSussex/status/8729576001...,Our reporter @JonHuntTV is in #Hastings tonigh...,@BBCSussex,BBC Sussex,United Kingdom,South East,NaN,NaN,NaN,Twitter,68.0,NaN,49073.0,103151.0,714.0,"[@JonHuntTV, @AmberRuddHR?]","[#Hastings, #SEVote, #bbcelection, #GE2017]"
826,872951646950744064,2017-06-08 23:01:00,http://twitter.com/Grasswire/status/8729516469...,@Grasswire @ChiOnwurah Conservatives win their...,@Grasswire,Grasswire,United States of America,NaN,NaN,NaN,NaN,Twitter,61.0,NaN,18624.0,182986.0,55349.0,"[@Grasswire, @ChiOnwurah]",[#GE2017]
914,872949710545453056,2017-06-08 22:53:00,http://twitter.com/EdwardTHardy/status/8729497...,Did @HackneyAbbott do the exit poll? #GE2017,@EdwardTHardy,Edward Hardy,United Kingdom,Greater London,London,NaN,NaN,Twitter,65.0,M,4322.0,42077.0,976.0,[@HackneyAbbott],[#GE2017]
525817,861149317167554560,2017-05-07 09:22:00,http://twitter.com/itvnews/status/861149317167...,RT @pestononsunday Responding to @TomBaldwin66...,@itvnews,ITV News,United Kingdom,NaN,NaN,NaN,NaN,Twitter,87.0,NaN,164830.0,1650333.0,1418.0,"[@pestononsunday, @TomBaldwin66's, @UKLabour's...","[#GE2017, #Peston]"
1620,872938513934819328,2017-06-08 22:08:00,http://twitter.com/BBCEssex/status/87293851393...,What will happen in the marginal Essex seat of...,@BBCEssex,BBC Essex,United Kingdom,East,NaN,NaN,NaN,Twitter,67.0,NaN,45687.0,40923.0,629.0,"[@JackieDP, @John4Thurrock17, @Tim_Aker, @Kevi...",[#GE2017]
2025,872934484097761280,2017-06-08 21:52:00,http://twitter.com/BBCEssex/status/87293448409...,Images from the Thurrock count where @SiDedman...,@BBCEssex,BBC Essex,United Kingdom,East,NaN,NaN,NaN,Twitter,67.0,NaN,45685.0,40922.0,629.0,"[@SiDedman, @JackieDP]",[#GE2017]
3565,872913597940346880,2017-06-08 20:29:00,http://twitter.com/TulipSiddiq/status/87291359...,RT @simonfrais Just voted in #GE2017 #VoteLabo...,@TulipSiddiq,Tulip Siddiq,United Kingdom,Greater London,London,NaN,NaN,Twitter,66.0,NaN,9159.0,34877.0,1125.0,"[@simonfrais, @TulipSiddiq]","[#GE2017, #VoteLabour, #PickTulip]"
